# **transformers 모델별 클래스 선택**

---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [1]:
# 설치된 transformers 버전 확인  -->(ex: Version: 4.55.4)
!pip show transformers | grep Version

Version: 4.57.1


In [3]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
%%capture
!pip install transformers==4.48.3

In [4]:
# 설치된 PyTorch 버전 확인  -->(ex: Version: 2.8.0+cu126 (+cu126-->CUDA 12.6 버전 지원))
!pip show torch | grep Version

# pip install torch==2.8.0

Version: 2.8.0+cu126


---
## **모델별 클래스 선택 예:**
- 참고 : http://bit.ly/42EuXjz

|모델구조	| 기본(헤드 없음)	| 태스크별 헤드(대표)|	 대표 모델 |
|---|---|---|---|
|인코더 기반 (Encoder-only)|	AutoModel	| 마스크드 LM<br> AutoModelForMaskedLM(문서/문장 분류)<br> AutoModelForSequenceClassification(토큰분류(NER)<br> AutoModelForTokenClassification(질의응답(SQuAD))<br> AutoModelForQuestionAnswering	| BERT, RoBERTa, DeBERTa, ELECTRA|
|디코더 기반 (Decoder-only, GPT 계열)|	AutoModel	|생성(다음 토큰 예측)<br> AutoModelForCausalLM(분류)<br>AutoModelForSequenceClassification(토큰분류)<br> AutoModelForTokenClassification	|GPT-2/Neo/NeoX/J, Llama/Llama-2/3, MPT, Phi |
|인코더–디코더 (Seq2Seq)|	AutoModel (드물게 사용)	|생성(번역·요약 등)<br> AutoModelForSeq2SeqLM (=…ForConditionalGeneration)분류(일부 지원)<br> AutoModelForSequenceClassification	|T5,BART, mBART, Marian |




---



## **모델별 클래스 사용 예제**
- 💡 **가능한 GPU 성능 좋은 환경에서 실행** (코랩 T4)

### **예제: 인코더 모델**

In [5]:
# 감정 분석 파이프라인
from transformers import pipeline

nlp = pipeline("sentiment-analysis")    # text-classification은 주로 인코더 모델이 사용됨
print(nlp("이 강의는 정말 재미있어요!"))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9887384176254272}]


### **예제: 디코더 모델**


In [1]:
from transformers import pipeline
# pipe = pipeline("text-generation", model="gpt2")   # 또는 Llama 계열
# pipe = pipeline("text-generation", model="Qwen/Qwen2.5-7B-Instruct")
# pipe = pipeline("text-generation", model="upstage/SOLAR-10.7B-Instruct-v1.0")
pipe = pipeline("text-generation", model="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", trust_remote_code=True)
pipe("오늘 날씨가 좋아서 기분이", max_new_tokens=50)


config.json: 0.00B [00:00, ?B/s]

configuration_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': '오늘 날씨가 좋아서 기분이 좋다. 오늘은 오랜만에 아내와 함께 오랜만에 카페에 가보았다#카페이름은 #카페명을모르는 #카페카페들은 #카페에서 #커피카페에서 #카페에서먹는커피 #카페에서 #카페'}]

In [3]:
print("=" * 60)
print("💬 채팅을 시작합니다. (종료: 'q' 입력 후 Enter)")
print("=" * 60)

# --- 채팅 반복 루프 ---
while True:
    # 키보드로 프롬프트 입력받기
    prompt = input("👤 당신: ")

    # 종료 조건 확인 (양쪽 공백 제거, 소문자 변환 후 'q'와 비교)
    if prompt.strip().lower() == 'q':
        print("✅ 채팅을 종료합니다.")
        break  # while 루프 탈출

    output = pipe(prompt, max_new_tokens=50)
    print("🤖 모델:", output[0]['generated_text'])


💬 채팅을 시작합니다. (종료: 'q' 입력 후 Enter)
👤 당신: 대한민국 수도는?
🤖 모델: 대한민국 수도는? 대한민국 수도는 서울입니다.
-

**질문 답변:**

### 질문: 대한민국의 수도는 어디인가요?

**답변:**
대한민국의 수도는 **서울**입니다.

### 세부 설명:
👤 당신: q
✅ 채팅을 종료합니다.


- 💡 **[참고] GatedRepoError 오류 발생시 해결방법** : 허깅페이스 사이트에서 해당 모델에 대한 Access권한 얻고 시행한다.
    - 허깅페이스 사이트에서 해당 모델 검색: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct
    - Expand to review and access 클릭
    - 개인정보 입력하고 Submit
    - Gated Repos Status 페이지에서 상태 확인 : https://huggingface.co/settings/gated-repos (ACCEPTED) 되어야함
    - 코랩에서 **세션 다시 시작하고**코드 실행

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.1-8B-Instruct"  # 예시(접근 권한 필요할 수 있음)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

prompt = "한국의 가을 단풍 여행지를 추천해줘."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


한국의 가을 단풍 여행지를 추천해줘. 가을 단풍 여행은 한국에서 가장 인기 있는 여행입니다.
가을 단풍 여행은 한국에서 가장 인기 있는 여행 중 하나입니다. 한국의 가을 단풍 여행지는 매년 10월에서 11월 사이에 단풍을 볼 수 있습니다. 이 기간 동안 한국의 많은 지역에서 단풍이 예술적으로 아름다운 색상을 띠고 있습니다. 가을 단풍 여행을 계획 중이신가요? 한국의 대표적인 가을 단풍 여행지를 알려드리겠습니다.

### 1. 남산
- **위치:** 서울시 종로구
- **가을 단풍:** 10월 15일 ~ 11월 15일
- **관광지:** 남산공원, 남산타워, 남산의등산로
- **특징:** 남산은 서울의 상징적인 랜드마크입니다. 가을 단풍 시기에는 남산 공원


### **예제: 인코더-디코더 모델**

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"  # ★ T5(인코더-디코더)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

def t5_sentiment(text):
    prompt = f"Classify the sentiment of the sentence as positive or negative.\nSentence: {text}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=4, do_sample=False)
    gen = tokenizer.decode(out[0], skip_special_tokens=True).strip().lower()
    if "positive" in gen:
        return "positive"
    elif "negative" in gen:
        return "negative"
    else:
        return f"unknown (모델 출력: {gen})"

for s in [
    "이 영화 진짜 재밌다! 최고의 주말이었다.",
    "스토리가 지루하고 배우 연기도 별로였어.",
    "음악은 좋았지만 전체적으로 평범했어."
]:
    print(s, "->", t5_sentiment(s))


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

이 영화 진짜 재밌다! 최고의 주말이었다. -> positive
스토리가 지루하고 배우 연기도 별로였어. -> negative
음악은 좋았지만 전체적으로 평범했어. -> negative


<!-- ### [미션] Phi 비교하기
1. 문학 작품(예: 헌법 전문 + 긴 소설 일부)을 통째로 넣어 보고, 요약을 비교하기
2. 회의록 20페이지 vs 100페이지를 넣어 두 버전에서 요약 결과 차이를 직접 확인
3. 숫자 세기, 문맥 요약 같은 단순 과제를 통해 차이를 눈으로 보여주기 -->



---

